In [1]:
import tensorflow as tf
import numpy as np
import os
train_paths = os.listdir('data/train/embedded_abstracts')
val_paths = os.listdir('data/val/val_embedded_abstract')

## Load the data
- embedded
- tokenized categories
- target

In [11]:
## Embedded
embedded_abstract = []
for path in train_paths:
    embedded_abstract.append(np.load('data/train/embedded_abstracts/'+path))
    if len(embedded_abstract) == 5: break

val_embedded_abstract = []
for path in val_paths:
    val_embedded_abstract.append(np.load('data/val/val_embedded_abstract/'+path))
    if len(val_embedded_abstract) == 5 :break

## tokenized
tokenized = np.load('data/train/tokenized.npy')

## target
target = np.load('data/train/target.npy')


In [12]:
embedded_abstract_padded = tf.keras.preprocessing.sequence.pad_sequences(embedded_abstract, padding='post', dtype=float)
val_embedded_abstract_padded = tf.keras.preprocessing.sequence.pad_sequences(val_embedded_abstract, padding='post', dtype=float)

In [17]:
def get_datasets(embedded_abstract, tokenized, target, batch_size=32):
    ## INPUTS.abstract
    embedded_abstract = tf.data.Dataset.from_tensor_slices(embedded_abstract)

    ## INPUTS.categories
    categories_dataset = tf.data.Dataset.from_tensor_slices(tokenized)
    
    ## INPUTS
    inputs_datasets = tf.data.Dataset.zip((embedded_abstract, categories_dataset))

    ## TARGETS
    target_dataset = tf.data.Dataset.from_tensor_slices(target)

    ## (INPUTS, TARGETS)
    datasets = tf.data.Dataset.zip((inputs_datasets, target_dataset))
    datasets = datasets.padded_batch(batch_size, padded_shapes= (([45, 512], [45]), []))
    
    return datasets

In [23]:
train_datasets = get_datasets(embedded_abstract=embedded_abstract_padded,tokenized=tokenized, target=target, batch_size=2)
#val_datasets = get_datasets(embedded_abstract=val_embedded_abstract, tokenized=val_tokenized, target=val_target)

In [25]:
for inputs, targets in train_datasets.take(2):
    print(f'inputs[abstract].shape = {inputs[0].shape}')
    print(f'inputs[categories].shape = {inputs[1].shape}')
    print(f'targets.shape = {targets.shape}')
    print()


inputs[abstract].shape = (2, 45, 512)
inputs[categories].shape = (2, 45)
targets.shape = (2,)

inputs[abstract].shape = (2, 45, 512)
inputs[categories].shape = (2, 45)
targets.shape = (2,)



# Train the model

In [26]:
from model import UseZeroClf

model = UseZeroClf(gru_units = 128, 
                   dense_units = [256, 128], 
                   category_vocab_size = tokenized.max(), 
                   emb_dim = 512)

In [27]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)

model.compile(optimizer=optimizer, loss=loss_object,
              metrics=['accuracy'])

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(
                monitor='loss', min_delta=0.001, patience=5, verbose=1, restore_best_weights=False
            )

history = model.fit(train_datasets, epochs=100, callbacks=[early_stop])